# Anna KaRNNa

In this notebook, we'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))

In [3]:
print(vocab)

['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
vocab_to_int = {c: i for i, c in enumerate(vocab)}

In [5]:
int_to_vocab = dict(enumerate(vocab))

In [6]:
int_to_vocab

{0: '\n',
 1: ' ',
 2: '!',
 3: '"',
 4: '$',
 5: '%',
 6: '&',
 7: "'",
 8: '(',
 9: ')',
 10: '*',
 11: ',',
 12: '-',
 13: '.',
 14: '/',
 15: '0',
 16: '1',
 17: '2',
 18: '3',
 19: '4',
 20: '5',
 21: '6',
 22: '7',
 23: '8',
 24: '9',
 25: ':',
 26: ';',
 27: '?',
 28: '@',
 29: 'A',
 30: 'B',
 31: 'C',
 32: 'D',
 33: 'E',
 34: 'F',
 35: 'G',
 36: 'H',
 37: 'I',
 38: 'J',
 39: 'K',
 40: 'L',
 41: 'M',
 42: 'N',
 43: 'O',
 44: 'P',
 45: 'Q',
 46: 'R',
 47: 'S',
 48: 'T',
 49: 'U',
 50: 'V',
 51: 'W',
 52: 'X',
 53: 'Y',
 54: 'Z',
 55: '_',
 56: '`',
 57: 'a',
 58: 'b',
 59: 'c',
 60: 'd',
 61: 'e',
 62: 'f',
 63: 'g',
 64: 'h',
 65: 'i',
 66: 'j',
 67: 'k',
 68: 'l',
 69: 'm',
 70: 'n',
 71: 'o',
 72: 'p',
 73: 'q',
 74: 'r',
 75: 's',
 76: 't',
 77: 'u',
 78: 'v',
 79: 'w',
 80: 'x',
 81: 'y',
 82: 'z'}

In [7]:
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [8]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [9]:
encoded[:100]

array([31, 64, 57, 72, 76, 61, 74,  1, 16,  0,  0,  0, 36, 57, 72, 72, 81,
        1, 62, 57, 69, 65, 68, 65, 61, 75,  1, 57, 74, 61,  1, 57, 68, 68,
        1, 57, 68, 65, 67, 61, 26,  1, 61, 78, 61, 74, 81,  1, 77, 70, 64,
       57, 72, 72, 81,  1, 62, 57, 69, 65, 68, 81,  1, 65, 75,  1, 77, 70,
       64, 57, 72, 72, 81,  1, 65, 70,  1, 65, 76, 75,  1, 71, 79, 70,  0,
       79, 57, 81, 13,  0,  0, 33, 78, 61, 74, 81, 76, 64, 65, 70])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [10]:
len(vocab)

83

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

> **Exercise:** Write the code for creating batches in the function below. The exercises in this notebook _will not be easy_. I've provided a notebook with solutions alongside this notebook. If you get stuck, checkout the solutions. The most important thing is that you don't copy and paste the code into here, **type out the solution code yourself.**

In [11]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps # sequence length * number of mini batches = window size (a matrix of characters)
    n_batches = len(arr)//characters_per_batch # batch = total length/ mini batch size
    
    # Keep only enough characters to make full batches
    arr = arr[:characters_per_batch*n_batches] # slice till the needed number of characters
    
    # Reshape into n_seqs rows
    arr = np.reshape(arr, (n_seqs,-1)) # number of rows = no of sequences
    
    for n in range(0, arr.shape[1], n_steps): # getting the number of columns needed in the reshaped array
        # The features
        x = arr[:,n:n+n_steps] # extract natch
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:,:-1],y[:,-1] = x[:,1:] ,x[:,0] #shift all the elements by one position and wrap last target with first element
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

### Note about "next" and "generator"

- https://stackoverflow.com/questions/1733004/python-next-
- https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do

In [12]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [13]:
x[1]

array([ 1, 57, 69,  1, 70, 71, 76,  1, 63, 71, 65, 70, 63,  1, 76, 71,  1,
       75, 76, 57, 81, 11,  3,  1, 57, 70, 75, 79, 61, 74, 61, 60,  1, 29,
       70, 70, 57, 11,  1, 75, 69, 65, 68, 65, 70, 63, 11,  1, 58, 77])

In [14]:
y[1]

array([57, 69,  1, 70, 71, 76,  1, 63, 71, 65, 70, 63,  1, 76, 71,  1, 75,
       76, 57, 81, 11,  3,  1, 57, 70, 75, 79, 61, 74, 61, 60,  1, 29, 70,
       70, 57, 11,  1, 75, 69, 65, 68, 65, 70, 63, 11,  1, 58, 77,  1])

In [15]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[31 64 57 72 76 61 74  1 16  0]
 [ 1 57 69  1 70 71 76  1 63 71]
 [78 65 70 13  0  0  3 53 61 75]
 [70  1 60 77 74 65 70 63  1 64]
 [ 1 65 76  1 65 75 11  1 75 65]
 [ 1 37 76  1 79 57 75  0 71 70]
 [64 61 70  1 59 71 69 61  1 62]
 [26  1 58 77 76  1 70 71 79  1]
 [76  1 65 75 70  7 76 13  1 48]
 [ 1 75 57 65 60  1 76 71  1 64]]

y
 [[64 57 72 76 61 74  1 16  0  0]
 [57 69  1 70 71 76  1 63 71 65]
 [65 70 13  0  0  3 53 61 75 11]
 [ 1 60 77 74 65 70 63  1 64 65]
 [65 76  1 65 75 11  1 75 65 74]
 [37 76  1 79 57 75  0 71 70 68]
 [61 70  1 59 71 69 61  1 62 71]
 [ 1 58 77 76  1 70 71 79  1 75]
 [ 1 65 75 70  7 76 13  1 48 64]
 [75 57 65 60  1 76 71  1 64 61]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`. This will be a scalar, that is a 0-D tensor. To make a scalar, you create a placeholder without giving it a size.

> **Exercise:** Create the input placeholders in the function below.

In [16]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32,shape=[batch_size, num_steps], name="inputs" )
    targets = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name="targets")
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32)
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

In [17]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character, so we want this layer to have size $C$, the number of classes/characters we have in our text.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$. 

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells. We get the LSTM output as a list, `lstm_output`. First we need to concatenate this whole list into one array with [`tf.concat`](https://www.tensorflow.org/api_docs/python/tf/concat). Then, reshape it (with `tf.reshape`) to size $(M * N) \times L$.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

> **Exercise:** Implement the output layer in the function below.

In [18]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns3
    x = tf.reshape(seq_output, shape= [-1 , in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal(shape=(in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(shape=(out_size)))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits=logits, name="predictions")
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

>**Exercise:** Implement the loss calculation in the function below.

In [19]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped =  tf.reshape(y_one_hot, shape=logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped))
    
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [20]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

> **Exercise:** Use the functions you've implemented previously and `tf.nn.dynamic_rnn` to build the network.

In [21]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size,num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN with tf.nn.dynamic_rnn 
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss,learning_rate, grad_clip)

## Hyperparameters

Here are the hyperparameters for the network.

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

In [23]:
batch_size = 256         # Sequences per batch
num_steps = 50          # Number of sequence steps per batch
lstm_size = 128         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.01    # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

> **Exercise:** Set the hyperparameters above to train the network. Watch the training loss, it should be consistently dropping. Also, I highly advise running this on a GPU.

In [24]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.4173...  0.4985 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.2426...  0.4215 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.6211...  0.4225 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 3.3750...  0.5000 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 3.3139...  0.4055 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.2830...  0.4140 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.2370...  0.4225 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.1930...  0.4160 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.2046...  0.4175 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.1977...  0.4170 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.1747...  0.4205 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.1677...  0.4050 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 1/20...  Training Step: 103...  Training loss: 2.4664...  0.5495 sec/batch
Epoch: 1/20...  Training Step: 104...  Training loss: 2.4473...  0.5235 sec/batch
Epoch: 1/20...  Training Step: 105...  Training loss: 2.4171...  0.5105 sec/batch
Epoch: 1/20...  Training Step: 106...  Training loss: 2.4018...  0.4660 sec/batch
Epoch: 1/20...  Training Step: 107...  Training loss: 2.4368...  0.5820 sec/batch
Epoch: 1/20...  Training Step: 108...  Training loss: 2.4236...  0.5780 sec/batch
Epoch: 1/20...  Training Step: 109...  Training loss: 2.4171...  0.4850 sec/batch
Epoch: 1/20...  Training Step: 110...  Training loss: 2.4143...  0.4845 sec/batch
Epoch: 1/20...  Training Step: 111...  Training loss: 2.3913...  0.4800 sec/batch
Epoch: 1/20...  Training Step: 112...  Training loss: 2.4165...  0.4880 sec/batch
Epoch: 1/20...  Training Step: 113...  Training loss: 2.4043...  0.4755 sec/batch
Epoch: 1/20...  Training Step: 114...  Training loss: 2.4235...  0.4630 sec/batch
Epoch: 1/20...  

Epoch: 2/20...  Training Step: 203...  Training loss: 2.1642...  0.4000 sec/batch
Epoch: 2/20...  Training Step: 204...  Training loss: 2.1404...  0.4015 sec/batch
Epoch: 2/20...  Training Step: 205...  Training loss: 2.1370...  0.4035 sec/batch
Epoch: 2/20...  Training Step: 206...  Training loss: 2.1074...  0.4035 sec/batch
Epoch: 2/20...  Training Step: 207...  Training loss: 2.1154...  0.4020 sec/batch
Epoch: 2/20...  Training Step: 208...  Training loss: 2.1121...  0.4045 sec/batch
Epoch: 2/20...  Training Step: 209...  Training loss: 2.1192...  0.4270 sec/batch
Epoch: 2/20...  Training Step: 210...  Training loss: 2.1193...  0.4420 sec/batch
Epoch: 2/20...  Training Step: 211...  Training loss: 2.1102...  0.4500 sec/batch
Epoch: 2/20...  Training Step: 212...  Training loss: 2.0830...  0.4665 sec/batch
Epoch: 2/20...  Training Step: 213...  Training loss: 2.1419...  0.4425 sec/batch
Epoch: 2/20...  Training Step: 214...  Training loss: 2.1020...  0.4750 sec/batch
Epoch: 2/20...  

Epoch: 2/20...  Training Step: 303...  Training loss: 1.9592...  0.4145 sec/batch
Epoch: 2/20...  Training Step: 304...  Training loss: 1.9830...  0.4185 sec/batch
Epoch: 2/20...  Training Step: 305...  Training loss: 1.9753...  0.4120 sec/batch
Epoch: 2/20...  Training Step: 306...  Training loss: 1.9727...  0.4325 sec/batch
Epoch: 2/20...  Training Step: 307...  Training loss: 1.9846...  0.4600 sec/batch
Epoch: 2/20...  Training Step: 308...  Training loss: 1.9601...  0.4705 sec/batch
Epoch: 2/20...  Training Step: 309...  Training loss: 1.9695...  0.4455 sec/batch
Epoch: 2/20...  Training Step: 310...  Training loss: 1.9625...  0.4220 sec/batch
Epoch: 3/20...  Training Step: 311...  Training loss: 2.0265...  0.4160 sec/batch
Epoch: 3/20...  Training Step: 312...  Training loss: 1.9786...  0.4160 sec/batch
Epoch: 3/20...  Training Step: 313...  Training loss: 1.9897...  0.4105 sec/batch
Epoch: 3/20...  Training Step: 314...  Training loss: 1.9669...  0.4200 sec/batch
Epoch: 3/20...  

Epoch: 3/20...  Training Step: 403...  Training loss: 1.8841...  0.4115 sec/batch
Epoch: 3/20...  Training Step: 404...  Training loss: 1.8870...  0.4110 sec/batch
Epoch: 3/20...  Training Step: 405...  Training loss: 1.8993...  0.4150 sec/batch
Epoch: 3/20...  Training Step: 406...  Training loss: 1.9135...  0.4335 sec/batch
Epoch: 3/20...  Training Step: 407...  Training loss: 1.9021...  0.4425 sec/batch
Epoch: 3/20...  Training Step: 408...  Training loss: 1.8833...  0.4215 sec/batch
Epoch: 3/20...  Training Step: 409...  Training loss: 1.9258...  0.4160 sec/batch
Epoch: 3/20...  Training Step: 410...  Training loss: 1.9169...  0.4195 sec/batch
Epoch: 3/20...  Training Step: 411...  Training loss: 1.8900...  0.4200 sec/batch
Epoch: 3/20...  Training Step: 412...  Training loss: 1.8885...  0.4170 sec/batch
Epoch: 3/20...  Training Step: 413...  Training loss: 1.9153...  0.4190 sec/batch
Epoch: 3/20...  Training Step: 414...  Training loss: 1.8884...  0.4145 sec/batch
Epoch: 3/20...  

...

Epoch: 18/20...  Training Step: 2673...  Training loss: 1.6272...  0.4050 sec/batch
Epoch: 18/20...  Training Step: 2674...  Training loss: 1.5956...  0.4085 sec/batch
Epoch: 18/20...  Training Step: 2675...  Training loss: 1.5636...  0.4030 sec/batch
Epoch: 18/20...  Training Step: 2676...  Training loss: 1.5969...  0.4045 sec/batch
Epoch: 18/20...  Training Step: 2677...  Training loss: 1.6092...  0.4045 sec/batch
Epoch: 18/20...  Training Step: 2678...  Training loss: 1.6032...  0.4085 sec/batch
Epoch: 18/20...  Training Step: 2679...  Training loss: 1.6102...  0.4325 sec/batch
Epoch: 18/20...  Training Step: 2680...  Training loss: 1.5924...  0.4345 sec/batch
Epoch: 18/20...  Training Step: 2681...  Training loss: 1.5944...  0.4080 sec/batch
Epoch: 18/20...  Training Step: 2682...  Training loss: 1.5949...  0.4015 sec/batch
Epoch: 18/20...  Training Step: 2683...  Training loss: 1.6093...  0.4040 sec/batch
Epoch: 18/20...  Training Step: 2684...  Training loss: 1.5859...  0.4015 se

Epoch: 18/20...  Training Step: 2771...  Training loss: 1.5692...  0.4065 sec/batch
Epoch: 18/20...  Training Step: 2772...  Training loss: 1.5746...  0.4020 sec/batch
Epoch: 18/20...  Training Step: 2773...  Training loss: 1.5679...  0.4060 sec/batch
Epoch: 18/20...  Training Step: 2774...  Training loss: 1.5911...  0.4035 sec/batch
Epoch: 18/20...  Training Step: 2775...  Training loss: 1.5855...  0.4045 sec/batch
Epoch: 18/20...  Training Step: 2776...  Training loss: 1.5972...  0.4020 sec/batch
Epoch: 18/20...  Training Step: 2777...  Training loss: 1.5602...  0.4000 sec/batch
Epoch: 18/20...  Training Step: 2778...  Training loss: 1.5808...  0.4045 sec/batch
Epoch: 18/20...  Training Step: 2779...  Training loss: 1.5938...  0.4055 sec/batch
Epoch: 18/20...  Training Step: 2780...  Training loss: 1.5642...  0.4040 sec/batch
Epoch: 18/20...  Training Step: 2781...  Training loss: 1.6188...  0.4290 sec/batch
Epoch: 18/20...  Training Step: 2782...  Training loss: 1.5969...  0.4380 se

Epoch: 19/20...  Training Step: 2869...  Training loss: 1.5705...  0.4230 sec/batch
Epoch: 19/20...  Training Step: 2870...  Training loss: 1.5791...  0.4055 sec/batch
Epoch: 19/20...  Training Step: 2871...  Training loss: 1.5898...  0.3995 sec/batch
Epoch: 19/20...  Training Step: 2872...  Training loss: 1.5804...  0.4045 sec/batch
Epoch: 19/20...  Training Step: 2873...  Training loss: 1.5712...  0.4025 sec/batch
Epoch: 19/20...  Training Step: 2874...  Training loss: 1.6027...  0.3985 sec/batch
Epoch: 19/20...  Training Step: 2875...  Training loss: 1.5696...  0.4020 sec/batch
Epoch: 19/20...  Training Step: 2876...  Training loss: 1.5914...  0.4020 sec/batch
Epoch: 19/20...  Training Step: 2877...  Training loss: 1.5339...  0.4005 sec/batch
Epoch: 19/20...  Training Step: 2878...  Training loss: 1.5842...  0.3990 sec/batch
Epoch: 19/20...  Training Step: 2879...  Training loss: 1.5714...  0.4010 sec/batch
Epoch: 19/20...  Training Step: 2880...  Training loss: 1.5886...  0.4045 se

Epoch: 20/20...  Training Step: 2967...  Training loss: 1.5897...  0.4000 sec/batch
Epoch: 20/20...  Training Step: 2968...  Training loss: 1.5647...  0.4055 sec/batch
Epoch: 20/20...  Training Step: 2969...  Training loss: 1.5839...  0.3990 sec/batch
Epoch: 20/20...  Training Step: 2970...  Training loss: 1.5685...  0.4020 sec/batch
Epoch: 20/20...  Training Step: 2971...  Training loss: 1.5522...  0.4340 sec/batch
Epoch: 20/20...  Training Step: 2972...  Training loss: 1.5735...  0.4315 sec/batch
Epoch: 20/20...  Training Step: 2973...  Training loss: 1.5865...  0.4060 sec/batch
Epoch: 20/20...  Training Step: 2974...  Training loss: 1.5450...  0.3985 sec/batch
Epoch: 20/20...  Training Step: 2975...  Training loss: 1.5448...  0.3975 sec/batch
Epoch: 20/20...  Training Step: 2976...  Training loss: 1.5644...  0.3995 sec/batch
Epoch: 20/20...  Training Step: 2977...  Training loss: 1.5787...  0.3990 sec/batch
Epoch: 20/20...  Training Step: 2978...  Training loss: 1.5748...  0.4075 se

Epoch: 20/20...  Training Step: 3065...  Training loss: 1.5792...  0.4030 sec/batch
Epoch: 20/20...  Training Step: 3066...  Training loss: 1.5742...  0.4005 sec/batch
Epoch: 20/20...  Training Step: 3067...  Training loss: 1.5876...  0.4020 sec/batch
Epoch: 20/20...  Training Step: 3068...  Training loss: 1.5916...  0.4010 sec/batch
Epoch: 20/20...  Training Step: 3069...  Training loss: 1.5876...  0.4035 sec/batch
Epoch: 20/20...  Training Step: 3070...  Training loss: 1.5978...  0.4005 sec/batch
Epoch: 20/20...  Training Step: 3071...  Training loss: 1.5753...  0.4020 sec/batch
Epoch: 20/20...  Training Step: 3072...  Training loss: 1.5868...  0.4125 sec/batch
Epoch: 20/20...  Training Step: 3073...  Training loss: 1.5718...  0.4315 sec/batch
Epoch: 20/20...  Training Step: 3074...  Training loss: 1.5773...  0.4210 sec/batch
Epoch: 20/20...  Training Step: 3075...  Training loss: 1.5791...  0.3995 sec/batch
Epoch: 20/20...  Training Step: 3076...  Training loss: 1.5783...  0.4020 se

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [31]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i3100_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2200_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2400_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2600_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i2800_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3000_l128.ckpt"
all_model_checkpoint_paths: "checkpoints\\i3100_l128.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [26]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [27]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [28]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints\\i3100_l128.ckpt'

In [29]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i3100_l128.ckpt
Farling words.

"I someous a merely, terling the comind that to atrimed the properition, all that she was not of this way of a sight what will are standing to a setter with
the crutch of any made one for
her, as thank that there are mile of same the things, what saw to have so had bueding, and that all the pastionals of the
mamp to the same shilling simply. But she had seemed in, seemed, that his methom, would be and will home on her standing the take him.

And so that to the part of the most, and had never have so work of her. And what had a stair to the case to stind of their stor hard of the corner that the tone of heavably seemed and who were alone that a sistle was a mistllowing him to have the some and taken, and
had this the cruth in his
words and she had and which thought, a corner,
here.

"I hovered it, went out who had simply simploved all the servant as and to be that all their had it she had never seen it

In [32]:
checkpoint = 'checkpoints/i200_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i200_l128.ckpt
Fared
and she somas to he her a where a stanss."
"
And hry since whiling with."

"Ye hand he had to seol, and
a to the was thes a woulds of the he searion, so would.
 "Why she
the sorted wout
sicare theild her the sang the womk, wince sime her sar the will.

"There her sing a merering
as she
with wer the wand
wat havis havont a was the sist whe the had her to tho hourd tans that a to that the cartel sarte all the what has to a matele the tomad there saire white and ans with tomith the sat the wist to wert was tine were we tansinch wer whish saess he he the was
and wish was
and seed, she thing
were and weins thrae his he thrite to shan wat
he with the shalle,
the
tonger was her and with has asing her sas the wous shing has she thit whot hat this, so that
seor a that and his to the have was wore sere hit andre has the shreeted hit hid
at and with the welt of he thing.

"I't had simed, whreatar the canded out to she him 

In [33]:
checkpoint = 'checkpoints/i600_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i600_l128.ckpt
Farleveng them would had that his
hand of the
hand who canting," said I has book all his time the down, so see of him and
what to to the same
and thinking that he had
stending, shound the peosent he
same and women.

"Yes so her tenciar of the passiansibut as at
the
some his tarted her, and and
thought
him, as he saw of
shem tratening of him he had hard this attaiched in any to the drece, as a feel well so she could not that that shooking of her that
the concition, and see the seraters would shandent her hit to his way
he had someing
all. And something were with he tell the sarsian, shook and him. The should so sometions, and the dead wound
but with the precitouress what he said house and than the souls, a day," said Levin had never as
the perpiston. "I she he done so
thought with the chest of her and seres, and they had stepted of her and houges, and the seculation at the histest of the decenesting that whon the
decus

In [34]:
checkpoint = 'checkpoints/i1200_l128.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/i1200_l128.ckpt
Farnts her, he could not thought the
commorsion and states he had so strook
was seet, and home that the peace and hour, so who were seeting a present the work her.

"Why, and he saw allothing to a lard to
have starched that we there though to he was as he could not
he had
shilt there, say of
the
did now of this stow him in his
happy, which as a still. Here all this."

His was a fromer work of the
doors, and any and the son, and his house in his husband of the conscicious of somether and was of the cit to his was at the
her say there were her. Afforce had the same
anyone of taken the called, and seemed help, and was
the with the
prepinoning of his strighted.
""Were what have thought towarkad, what were thinking at into to who convinting
the care of that
he she saw him to the
decate her head. That same to all a same. "A nothing the some always with the prathen with the some torcially stopped
that when he heard a stigh 